In [ ]:
import os
os.chdir("/gpfs/home/asun/jin_lab/perturbench/0_datasets")
print(os.getcwd())

import sys
sys.path.append(os.path.abspath('..'))

In [ ]:
import scanpy as sc
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
boli_wb = sc.read_h5ad("/gpfs/home/asun/processed.h5ad")
boli_ctx = sc.read_h5ad('/gpfs/home/asun/jin_lab/perturbench/0_datasets/boli_ctx_scprocess_no+ctrl.h5ad')


In [ ]:
sc.pp.normalize_total(boli_wb)
sc.pp.log1p(boli_wb)

In [ ]:
boli_wb_filt = boli_wb[:, boli_wb.var_names.isin(boli_ctx.var_names)].copy()

In [ ]:
import numpy as np
from scipy import sparse

cell_type_col   = "cell_type"      # e.g. "class_name" / "mmc_label"
cell_type_value = "030 L6 CT CTX Glut"  # <-- set this
condition_col   = "condition"

mask = (boli_wb_filt.obs[condition_col] == "TBR1+ctrl") & \
       (boli_wb_filt.obs[cell_type_col] == cell_type_value)

X = boli_wb_filt.X
avg_p = X[mask].mean(axis=0)
avg_p = avg_p.A1 if sparse.issparse(avg_p) else np.asarray(avg_p).ravel()

cell_type_col   = "cell_type"      # e.g. "class_name" / "mmc_label"
cell_type_value = "030 L6 CT CTX Glut"  # <-- set this
condition_col   = "condition"

mask = (boli_wb_filt.obs[condition_col] == "ctrl") & \
       (boli_wb_filt.obs[cell_type_col] == cell_type_value)

X = boli_wb_filt.X
avg_ctrl = X[mask].mean(axis=0)
avg_ctrl = avg_ctrl.A1 if sparse.issparse(avg_ctrl) else np.asarray(avg_ctrl).ravel()

pseudocount = 0.1
avg_p = np.log2(avg_p + pseudocount)
avg_ctrl = np.log2(avg_ctrl + pseudocount)

logfc = avg_p - avg_ctrl

# Create dataframe with gene names as columns and logFC as a single row
gene_names = boli_wb_filt.var["gene_name"]  # or boli_wb_filt.var.index
logfc_df = pd.DataFrame([logfc], columns=gene_names)

print(logfc_df)

In [ ]:
import numpy as np
from scipy import sparse

cell_type_col   = "cell_type"      # e.g. "class_name" / "mmc_label"
cell_type_value = "L6_CT_CTX"  # <-- set this
condition_col   = "condition"

mask = (boli_ctx.obs[condition_col] == "TBR1") & \
       (boli_ctx.obs[cell_type_col] == cell_type_value)

X = boli_ctx.X
avg_p = X[mask].mean(axis=0)
avg_p = avg_p.A1 if sparse.issparse(avg_p) else np.asarray(avg_p).ravel()

cell_type_col   = "cell_type"      # e.g. "class_name" / "mmc_label"
cell_type_value = "L6_CT_CTX"  # <-- set this
condition_col   = "condition"

mask = (boli_ctx.obs[condition_col] == "ctrl") & \
       (boli_ctx.obs[cell_type_col] == cell_type_value)

X = boli_ctx.X
avg_ctrl = X[mask].mean(axis=0)
avg_ctrl = avg_ctrl.A1 if sparse.issparse(avg_ctrl) else np.asarray(avg_ctrl).ravel()

pseudocount = 0.1
avg_p = np.log2(avg_p + pseudocount)
avg_ctrl = np.log2(avg_ctrl + pseudocount)

logfc = avg_p - avg_ctrl

# Create dataframe with gene names as columns and logFC as a single row
gene_names = boli_ctx.var["gene_name"]  # or boli_wb_filt.var.index
logfc_ctx_df = pd.DataFrame([logfc], columns=gene_names)

print(logfc_ctx_df)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr
from adjustText import adjust_text

# Assuming you have logfc_df_ctx and logfc_df_wb
# Both should have the same gene columns

logfc_df_wb = logfc_df
logfc_df_ctx = logfc_ctx_df

# Get common genes between the two datasets
common_genes = logfc_df_ctx.columns.intersection(logfc_df_wb.columns)

# Extract logFC values for common genes
x_vals = logfc_df_ctx[common_genes].values[0]
y_vals = logfc_df_wb[common_genes].values[0]

# Calculate correlation
pearson_r, p_value = pearsonr(x_vals, y_vals)

# Create the plot
fig, ax = plt.subplots(figsize=(10, 10))

# Scatter plot
ax.scatter(x_vals, y_vals, alpha=0.5, color="lightgray", s=20)

# Identity line
min_val = min(x_vals.min(), y_vals.min())
max_val = max(x_vals.max(), y_vals.max())
ax.plot([min_val, max_val], [min_val, max_val], 'k--', alpha=0.5, label='y=x')

# Labels and title
ax.set_xlabel("LogFC (CTX)", fontsize=14)
ax.set_ylabel("LogFC (WB)", fontsize=14)
ax.set_title(f"LogFC Comparison: CTX vs WB\nPearson r = {pearson_r:.3f} (p = {p_value:.2e})", fontsize=16)
ax.grid(True, alpha=0.3)

# Highlight top/bottom genes
top10_ctx_idx = np.argsort(x_vals)[-10:]
bottom10_ctx_idx = np.argsort(x_vals)[:10]
top10_wb_idx = np.argsort(y_vals)[-10:]
bottom10_wb_idx = np.argsort(y_vals)[:10]

# Combine indices for labeling (genes that are extreme in either dataset)
genes_to_label = set(top10_ctx_idx).union(set(bottom10_ctx_idx)).union(
                     set(top10_wb_idx)).union(set(bottom10_wb_idx))

# Color coding
texts = []
for idx in genes_to_label:
    if idx in top10_ctx_idx or idx in top10_wb_idx:
        color = "steelblue"
    else:
        color = "firebrick"
    
    # Scatter the highlighted points
    ax.scatter(x_vals[idx], y_vals[idx], color=color, s=50, alpha=0.8, edgecolors='black', linewidths=0.5)
    
    # Add text label
    texts.append(
        ax.text(
            x_vals[idx], y_vals[idx], common_genes[idx],
            fontsize=10,
            color=color,
            alpha=0.9
        )
    )

# Adjust text to avoid overlap
adjust_text(texts, arrowprops=dict(arrowstyle='->', color='gray', lw=0.5), ax=ax)

plt.tight_layout()
plt.show()